# Azure OpenAI에서 Function Calling 사용 방법
이 파일은 현재 GPT 모델의 성능을 확장하기 위하여 Function Calling을 Chat Completion API에 사용하는 방법을 보여줍니다. GPT 모델은 본래 외부 시스템, 데이터베이스 또는 파일과의 실시간 상호 작용을 지원하지 않습니다. 그러나 Function Calling을 사용하면 가능해 집니다. 이 자료는 아래의 URL을 한글로 번역해 놓은 자료입니다.
- 원문 URL: https://github.com/Azure-Samples/openai/tree/main/Basic_Samples/Functions

## Overview
`functions`은 Chat Completion API의 옵션으로 제공하는 파라미터이며, 함수의 명세를 제공하는 데 사용할 수 있습니다. 이를 통해 모델은 사용자가 제공한 함수 명세로부터 함수에 필요한 인수(arguments)를 생성할 수 있습니다.

참고: API는 함수 호출을 실행하지 않습니다. 출력 인수를 사용하여 함수 호출을 실행하는 것은 개발자가 만들어야 합니다.

## Setup

In [1]:
# 참고: 최신 버전의 OpenAI Python 라이브러리를 설치 및/또는 업그레이드를 권장합니다.
%pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import json
import os

from dotenv import load_dotenv
load_dotenv()

# Azure OpenAI resource 정보를 설정합니다.
openai.api_type     = os.getenv("OPENAI_API_TYPE")
openai.api_key      = os.getenv("OPENAI_API_KEY")
openai.api_base     = os.getenv("OPENAI_API_BASE")
openai.api_version  = os.getenv("OPENAI_API_VERSION")   # API 버전은 "2023-07-01-preview" 부터 사용 가능합니다.
deployment_id       = os.getenv("DEPLOYMENT_NAME")      # Azure OpenAI 리소스의 배포 이름입니다. gpt-35-turbo (0613) 또는 gpt-4 (0613) 버전 이상부터 지원합니다.

## 1. `Function Call` 테스트

이 코드는 사용자 쿼리와 functions 매개변수에 정의된 함수 집합을 사용하여 모델을 호출합니다. 그런 다음 모델은 Function 호출 여부를 선택할 수 있습니다. 함수가 호출되면 콘텐츠는 문자열화된 JSON 객체에 넣습니다. 만들어야 하는 Function Call 과 인수(arguments)는 `response['choices'][0]['message']['function_call']`에 위치합니다.

In [3]:
def get_function_call(messages, function_call = "auto"):
    # Define the functions to use
    functions = [
        {
            "name": "get_current_weather",
            "description": "해당 지역의 현재 날씨를 알려줘.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "지역(도시) 이름, e.g. 서울시 동작구",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["섭씨", "화씨"]},
                },
                "required": ["location", "unit"],
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = openai.ChatCompletion.create(
        deployment_id = deployment_id,
        messages=messages,
        functions=functions,
        function_call=function_call, 
    )
    return response

### 특정 `Function`을 강제로 사용하거나 사용하지 않도록 설정
`functions` 매개변수(parameter)의 값을 변경하여 모델이 사용할 function을 결정하도록 허용하거나, 모델이 특정 function을 사용하도록 강제하거나, 모델이 function을 사용하지 않도록 강제할 수 있습니다.

In [4]:
first_message = [{"role": "user", "content": "서울 동작구의 날씨는 어때?"}]
# 'auto' : Let the model decide what function to call
print("모델이 호출하려는 function을 자동으로 결정:")
response = get_function_call(first_message, "auto")
print (response)
# print (json.dumps(response['choices'][0]['message']['function_call'], ensure_ascii=False, indent=4))

# 'none' : Don't call any function 
print("모델이 어떤 function도 호출하지 않음:")
response = get_function_call(first_message, "none")
print (response)

# force a specific function call
print("모델이 강제로 지정한 function을 호출:")
response = get_function_call(first_message, function_call={"name": "get_current_weather"})
print (response)
# print (json.dumps(response['choices'][0]['message']['function_call'], ensure_ascii=False, indent=4))


모델이 호출하려는 function을 자동으로 결정:
{
  "id": "chatcmpl-7feNh7Nyuak1MgLrYQefSYsqIfFNC",
  "object": "chat.completion",
  "created": 1690161029,
  "model": "gpt-35-turbo",
  "prompt_annotations": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "function_call",
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"\uc11c\uc6b8\uc2dc \ub3d9\uc791\uad6c\",\n  \"unit\": \"\uc12d\uc528\"\n}"
        }
      },
      "content_filter_results": {}

## 2. functions 정의
이제 함수로 작업하는 방법을 알았으므로 코드에서 몇 가지 함수를 정의하여 함수를 사용하는 프로세스를 끝까지 살펴보겠습니다.

### `Function #1`: Get current time

In [5]:
import pytz
from datetime import datetime

def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%Y-%m-%d, %H:%M:%S")

        return current_time
    except:
        return "죄송합니다. 해당 지역의 TimeZone을 찾을 수 없습니다."

In [6]:
get_current_time("Asia/Seoul")

'2023-07-24, 10:10:42'

### `Function #2`: Get stock market data
이 예제에서는 쉬운 설명을 위해 일부 주식 시장 데이터를 하드 코딩한 csv 파일 형태로 제공하지만, 코드를 편집하여 API를 호출하여 실시간 데이터를 활용할 수 있습니다.

In [7]:
import pandas as pd
import json

def get_stock_market_data(index):
    available_indices = ["S&P 500", "NASDAQ Composite", "Dow Jones Industrial Average", "Financial Times Stock Exchange 100 Index"]

    if index not in available_indices:
        return "Invalid index. Please choose from 'S&P 500', 'NASDAQ Composite', 'Dow Jones Industrial Average', 'Financial Times Stock Exchange 100 Index'."

    # Read the CSV file
    data = pd.read_csv('./data/stock_data.csv')
    
    # Filter data for the given index
    data_filtered = data[data['Index'] == index]

    # Remove 'Index' column
    data_filtered = data_filtered.drop(columns=['Index'])

    # Convert the DataFrame into a dictionary
    hist_dict = data_filtered.to_dict()

    for key, value_dict in hist_dict.items():
        hist_dict[key] = {k: v for k, v in value_dict.items()}

    return json.dumps(hist_dict, indent=4)


In [8]:
print(get_stock_market_data("NASDAQ Composite"))

{
    "Date": {
        "2": "2023-07-12",
        "3": "2023-07-13"
    },
    "Open": {
        "2": 14000.65,
        "3": 14100.11
    },
    "High": {
        "2": 14200.06,
        "3": 14250.0
    },
    "Low": {
        "2": 13800.08,
        "3": 14000.67
    },
    "Close": {
        "2": 14100.44,
        "3": 14050.81
    },
    "Volume": {
        "2": 4000000,
        "3": 4200000
    }
}


### `Function #3`: Calculator
계산 기능은 math 라이브러리를 이용하여 정확하게 계산하도록 유도합니다.

In [9]:
import math

def calculator(num1, num2, operator):
    if operator == '+':
        return str(num1 + num2)
    elif operator == '-':
        return str(num1 - num2)
    elif operator == '*':
        return str(num1 * num2)
    elif operator == '/':
        return str(num1 / num2)
    elif operator == '**':
        return str(num1 ** num2)
    elif operator == 'sqrt':
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [10]:
print(calculator(5, 5, '+'))

10


## 3. GPT를 사용한 `Function` 호출

`Function Calling`을 위한 단계: 

1. 사용자 쿼리와 functions 매개변수(parameter)에 정의된 함수 집합을 사용하여 모델을 호출합니다.
2. 모델은 함수 호출을 선택할 수 있습니다. 콘텐츠는 사용자 지정 스키마를 준수하는 문자열화된 JSON 객체가 됩니다(참고: 모델이 잘못된 JSON 또는 환각(hallucination) 매개변수를 생성할 수 있음).
3. 코드에서 문자열을 JSON으로 구문 분석합니다. 제공된 인수가 있는 경우 해당 인수로 함수를 호출합니다.
4. 함수 응답을 새 메시지로 추가하여 모델을 다시 호출하고 모델이 결과를 사용자에게 다시 요약하도록 합니다.

### 3.1 모델이 호출 방법을 알 수 있도록 함수 설명

In [11]:
functions = [
        {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            # "description": "이 지역의 현재 시간을 알려줘.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like Asia/Seoul, America/New_York, Asia/Bangkok, Europe/London",
                        # "description": "location 이름. pytz는 해당 위치의 시간대를 가져오는 데 사용됩니다. location 이름은 Asia/Seoul, America/New_York, Asia/Bangkok, Europe/London과 같은 형식이어야 합니다.",
                    }
                },
                "required": ["location"],
            },
        },
        {
            "name": "get_stock_market_data",
            "description": "Get the stock market data for a given index",
            # "description": "주어진 인덱스에 대한 주식 시장 데이터 가져오기",
            "parameters": {
                "type": "object",
                "properties": {
                    "index": {
                        "type": "string",
                        "enum": ["S&P 500", "NASDAQ Composite", "Dow Jones Industrial Average", "Financial Times Stock Exchange 100 Index"]},
                },
                "required": ["index"],
            },    
        },
        {
            "name": "calculator",
            "description": "A simple calculator used to perform basic arithmetic operations",
            # "description": "기본 산술 연산을 수행하는 데 사용되는 간단한 계산기",
            "parameters": {
                "type": "object",
                "properties": {
                    "num1": {"type": "number"},
                    "num2": {"type": "number"},
                    "operator": {"type": "string", "enum": ["+", "-", "*", "/", "**", "sqrt"]},
                },
                "required": ["num1", "num2", "operator"],
            },
        }
    ]

available_functions = {
            "get_current_time": get_current_time,
            "get_stock_market_data": get_stock_market_data,
            "calculator": calculator,
        } 

### 3.2 function call을 검증하는 helper 함수 정의
모델이 잘못된 function call을 생성할 수 있으므로 function의 유효성을 검사하는 것이 중요합니다. 여기서는 사용 사례에 대해 더 복잡한 유효성 검사를 적용할 수 있지만 함수 호출의 유효성을 검사하는 간단한 helper 함수를 정의합니다.

In [12]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [13]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    response_message = response["choices"][0]["message"]


    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        fuction_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(fuction_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = fuction_to_call(**function_args)
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        # for message in messages:
        #     print(message)
        # print()
        print(json.dumps(messages, ensure_ascii=False, indent=4))
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

In [14]:
messages = [{"role": "user", "content": "시애틀의 현재 시간은?"}]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
print(assistant_response['choices'][0]['message'])
json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)

Recommended Function call:
{
  "name": "get_current_time",
  "arguments": "{\n\"location\": \"America/Los_Angeles\"\n}"
}

Output of function call:
2023-07-23, 18:10:43

Messages in second request:
[
    {
        "role": "user",
        "content": "시애틀의 현재 시간은?"
    },
    {
        "role": "assistant",
        "name": "get_current_time",
        "content": "{\n\"location\": \"America/Los_Angeles\"\n}"
    },
    {
        "role": "function",
        "name": "get_current_time",
        "content": "2023-07-23, 18:10:43"
    }
]

{
  "role": "assistant",
  "content": "\ud604\uc7ac \uc2dc\uc560\ud2c0\uc758 \uc2dc\uac04\uc740 2023\ub144 7\uc6d4 23\uc77c \uc624\ud6c4 6\uc2dc 10\ubd84 43\ucd08\uc785\ub2c8\ub2e4."
}


'"현재 시애틀의 시간은 2023년 7월 23일 오후 6시 10분 43초입니다."'

## 4. 동시에 다수의 functions을 호출
여러 function call을 함께 연결하여 원하는 결과를 얻을 수 있습니다. 여러 함수 호출이 가능하도록 위의 `run_conversation()` 함수를 수정했습니다.

In [15]:
def run_multiturn_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
        temperature=0
    )

    # Step 2: check if GPT wanted to call a function
    while response["choices"][0]["finish_reason"] == 'function_call':
        response_message = response["choices"][0]["message"]
        # print("Recommended Function call:")
        # print(response_message.get("function_call"))
        # print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        fuction_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(fuction_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = fuction_to_call(**function_args)
        
        # print("Output of function call:")
        # print(function_response)
        # print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in next request:")
        # for message in messages:
        #     print(message)
        print(json.dumps(messages, ensure_ascii=False, indent=4))
        print()

        response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id,
            function_call="auto",
            functions=functions,
            temperature=0
        )  # get a new response from GPT where it can see the function response

    return response

In [16]:
# Can add system prompting to guide the model to call functions and perform in specific ways
next_messages = [{"role": "system", "content": "Assistant is a helpful assistant that helps users get answers to questions. Assistant has access to several tools and sometimes you may need to call multiple tools in sequence to get answers for your users."}]
next_messages.append({"role": "user", "content": "How much did S&P 500 change between July 12 and July 13? Use the calculator."})
# next_messages = [{"role": "system", "content": "너는 사용자의 질문에 대한 답을 얻을 수 있도록 도와주는 유용한 어시스턴트입니다. 어시스턴트는 여러 도구에 액세스할 수 있으며 경우에 따라 사용자의 답변을 얻기 위해 여러 도구를 순서대로 호출해야 할 수도 있습니다."}]
# next_messages.append({"role": "user", "content": "S&P 500은 7월 12일에서 7월 13일에 얼만큼 변경되었습니까? 계산기를 사용하십시오."})

assistant_response = run_multiturn_conversation(next_messages, functions, available_functions, deployment_id)
print("Final Response:")
print(assistant_response["choices"][0]["message"])
print(json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False))
print("Conversation complete!")  

Messages in next request:
[
    {
        "role": "system",
        "content": "Assistant is a helpful assistant that helps users get answers to questions. Assistant has access to several tools and sometimes you may need to call multiple tools in sequence to get answers for your users."
    },
    {
        "role": "user",
        "content": "How much did S&P 500 change between July 12 and July 13? Use the calculator."
    },
    {
        "role": "assistant",
        "name": "get_stock_market_data",
        "content": "{\n  \"index\": \"S&P 500\"\n}"
    },
    {
        "role": "function",
        "name": "get_stock_market_data",
        "content": "{\n    \"Date\": {\n        \"0\": \"2023-07-12\",\n        \"1\": \"2023-07-13\"\n    },\n    \"Open\": {\n        \"0\": 4300.25,\n        \"1\": 4325.55\n    },\n    \"High\": {\n        \"0\": 4350.32,\n        \"1\": 4350.0\n    },\n    \"Low\": {\n        \"0\": 4200.2,\n        \"1\": 4300.98\n    },\n    \"Close\": {\n        \"0\